In [ ]:
import sqlite3

import pandas as pd

In [ ]:
with sqlite3.connect("../data/survey.db") as connection:
    df = pd.read_sql(sql="SELECT Site.lat, Site.long FROM Site;", con=connection)

In [ ]:
df

In [ ]:
def get_name(connection, person_identifier):
    query = f"SELECT personal || ' ' || family FROM Person WHERE id='{person_identifier}';"
    name, = (pd.read_sql(sql=query, con=connection)
               .iloc[0]
               .values)
    return name


In [ ]:
with sqlite3.connect("../data/survey.db") as connection:
    name = get_name(connection, "dyer")

In [ ]:
name

In [ ]:
def get_name(connection, person_identifier):
    query = f"SELECT personal || ' ' || family FROM Person WHERE id=?;"
    name, = (pd.read_sql(sql=query, con=connection, params=[person_identifier])
               .iloc[0]
               .values)
    return name


In [ ]:
with sqlite3.connect("../data/survey.db") as connection:
    name = get_name(connection, "dyer")

In [ ]:
name

## Filling a Table vs. Printing Values

In [ ]:
import numpy as np

prng = np.random.RandomState(42)
numbers = prng.uniform(low=10.0, high=25.0, size=1000000)
df = pd.DataFrame.from_dict({"reading": numbers})

In [ ]:
%%timeit

with sqlite3.connect("../data/original.db") as connection:
    df.to_sql(name="Pressure",
              con=connection,
              if_exists="replace",
              index=False,
              dtype={"reading": "real"})

In [ ]:
%timeit df.to_csv("../data/original.csv", index=False)

**Takeaway**: Writing to SQL DB is significantly faster than writing to disk!

## Filtering in SQL vs. Filtering in Python

In [ ]:
%%timeit

with sqlite3.connect("../data/original.db") as connection:
    df = pd.read_sql(sql="SELECT * FROM Pressure;", con=connection)

with sqlite3.connect("../data/backup.db") as connection:
    (df[df["reading"] > 20.0]
       .to_sql(name="Pressure",
               con=connection,
               if_exists="replace",
               index=False,
               dtype={"reading": "real"}))

In [ ]:
%%timeit

with sqlite3.connect("../data/original.db") as connection:
    df = pd.read_sql(sql="SELECT * FROM Pressure WHERE reading > 20.0;", con=connection)

with sqlite3.connect("../data/backup.db") as connection:
    df.to_sql(name="Pressure",
              con=connection,
              if_exists="replace",
              index=False,
              dtype={"reading": "real"})

**Takeaway:** Pushing computation into the DB will improve performance!